#Data Augmentation Iteration 1

Install opendatasets

In [1]:
!pip install opendatasets

Open the dataset from Kaggle. Requires username and key, see README.md for more details

In [3]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/jonathanoheix/face-expression-recognition-dataset/data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: johnnyplays
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/jonathanoheix/face-expression-recognition-dataset


Delete redundant image folder

In [4]:
import shutil
shutil.rmtree("/content/face-expression-recognition-dataset/images/images")

Establish training and testing set

In [5]:
train_set = "/content/face-expression-recognition-dataset/images/train"
test_set = "/content/face-expression-recognition-dataset/images/validation"

Function to create dataframe

In [6]:
def dataframe(direct):
    image_paths = []
    labels = []
    for label in os.listdir(direct):
        for filename in os.listdir(os.path.join(direct, label)):
            image_paths.append(os.path.join(direct, label, filename))
            labels.append(label)

            # Check if it's an original image (not translated)
            if not filename.endswith("_translated.jpg"):
                # Add translated version if it exists
                translated_filename = f"{filename[:-4]}_translated.jpg"
                translated_path = os.path.join(direct, label, translated_filename)
                if os.path.exists(translated_path):
                    image_paths.append(translated_path)
                    labels.append(label)  # Same label as original

        print(label, "completed")
    return image_paths, labels

Create dataframe for training set

In [10]:
import pandas as pd
import os

In [11]:
train = pd.DataFrame()
train['image'], train['label'] = dataframe(train_set)

neutral completed
disgust completed
sad completed
fear completed
surprise completed
happy completed
angry completed


Create dataframe for testing set

In [12]:
test = pd.DataFrame()
test['image'], test['label'] = dataframe(test_set)

neutral completed
disgust completed
sad completed
fear completed
surprise completed
happy completed
angry completed


Function for creating translated images for data augmentation

In [13]:
def translate_image(image, shift_x, shift_y):
    """Translates an image by the given shift values.

    Args:
        image: The input image as a NumPy array.
        shift_x: The shift in the x-direction (horizontal).
        shift_y: The shift in the y-direction (vertical).

    Returns:
        The translated image as a NumPy array.
    """

    M = np.float32([[1, 0, shift_x], [0, 1, shift_y]])
    translated_image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
    return translated_image

Implement the translation function on training set

In [17]:
import numpy as np
import cv2
import os
from tqdm import tqdm

In [18]:
training_dir = "/content/face-expression-recognition-dataset/images/train"  # Update with your validation data path

# Define translation parameters
shift_range = 5  # Maximum translation in pixels (adjust as needed)

for emotion_folder in os.listdir(training_dir):
    emotion_path = os.path.join(training_dir, emotion_folder)
    for filename in tqdm(os.listdir(emotion_path)):
        image_path = os.path.join(emotion_path, filename)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale

        # Generate random translation values
        shift_x = np.random.randint(-shift_range, shift_range + 1)
        shift_y = np.random.randint(-shift_range, shift_range + 1)

        # Translate the image
        translated_image = translate_image(image, shift_x, shift_y)

        # Save translated image with a new name
        new_filename = f"{filename[:-4]}_translated.jpg"  # Example naming
        new_image_path = os.path.join(emotion_path, new_filename)
        cv2.imwrite(new_image_path, translated_image)

print("Image translation completed.")

100%|██████████| 3993/3993 [00:01<00:00, 3348.19it/s]

Image translation completed.
